#  Track Analysis

In order to properly analyze our GPX tracks, we need to put all of them into the same _frame-of-reference_.   This is accomplished by the following algorithm...

* Create a set of variables with the initialized values
    * Current Coordinate = Starting Coordinate
        * $P_{\textrm{cur}} = P_{\textrm{start}}$
        * **NOTE:** Starting coordinate $P_{\textrm{start}}$ is defined below as a fixed variable
    * Distance Threshold = Small distance in meters
        * $d_{\textrm{thresh}} = 20 meters$
        * This is the distance between a possible point and the current point for consideration.
    * Step Distance = Small distance in meters.
        * $d_{\textrm{step}} = 20 meters$
        * This is the distance that the route will jump using the average angle
    * Waypoint list = empty array
        * $\hat{P}_{\textrm{waypoints}} = []$

## Step 0: Globals

In [1]:
database_path = 'bike_data.db'

start_coord = (39.5989743, -104.8609468)
end_coord   = (39.75428108249532, -105.00085402872664)

epsg_code = 32613

## Step 1: Import Required Libraries

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from pyproj import CRS, Proj, Transformer
from prinpy.local import CLPCG
from prinpy.glob import NLPCA
import re, numpy as np

### Step 1.1: Setup the database.

In [3]:
# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT datasetId FROM point_list', conn)

### Step 1.2: Setup UTM Coordinate Projection API

In [4]:
#  Setup the Projection Transformer
crs = CRS.from_epsg( epsg_code )
proj_dd2utm = Transformer.from_crs(crs.geodetic_crs, crs)
proj_utm2dd = Transformer.from_crs(crs, crs.geodetic_crs)
utm_zone = int(re.findall("\d+", crs.utm_zone)[0])
print('UTM Grid Zone: {}'.format(utm_zone))

(easting,northing) = proj_dd2utm.transform( start_coord[0], start_coord[1] )
start_coord_utm = np.array( [easting, northing], np.float64 )

UTM Grid Zone: 13


## Step 2: Build Routes for Each Sector

This algorithm works best with small batches of points.  It's important to work sector-by-sector to build the final route. 

In [ ]:
def Process_Sector( sector_id, start_point_utm, e_max ):
    
    #  Load coordinates
    sql_query = 'SELECT * FROM point_list WHERE sector_id is {} ORDER BY timestamp'.format( sector_id )
    points = pd.read_sql_query( sql_query, conn )
    utm_points = points.loc[:,['easting','northing']]
    print(sql_query)
    
    #  Create solver
    cl = CLPCG()
    
    # the fit() method calculates the principal curve
    # e_max is determined through trial and error as of
    # now, but aim for about 1/2 data error and adjust from
    # there.
    x_vals = utm_points.loc[:,'easting']
    y_vals = utm_points.loc[:,'northing']

    x_vals = pd.concat([pd.Series([easting]), x_vals])
    y_vals = pd.concat([pd.Series([northing]), y_vals])

    x_min = x_vals.min()
    x_max = x_vals.max()
    y_min = y_vals.min()
    y_max = y_vals.max()

    x_vals_norm = 2 * (x_vals - x_min) / (x_max - x_min) - 1
    y_vals_norm = 2 * (y_vals - y_min) / (y_max - y_min) - 1

    cl.fit( x_vals_norm.to_numpy(), 
            y_vals_norm.to_numpy(),
            e_max = 0.05 )  # CLPCG.fit() to fit PC

    spline_pts_local = cl.fit_points   # fitted points with PC that spline is passed through
    spline_x_local = ( (spline_pts_local[:,0] + 1.0)/2.0 ) * ( x_max - x_min ) + x_min
    spline_y_local = ( (spline_pts_local[:,1] + 1.0)/2.0 ) * ( y_max - y_min ) + y_min

    # Convert Spline Points to Lat/Lon
    prin_lats_local, prin_lons_local = proj_utm2dd.transform( spline_x_local, spline_y_local )
    print('Spline has {} coordinates'.format(len(spline_x_local)))
    
    return spline_x


## Step 4: Compute the starting point

The first point is weird cause it establishes the anchor points within our master route.

In [7]:
avg_pt_utm = np.array([0, 0], dtype=np.float64)
temp_angles = []
pt_count = 0

current_dataset_idx = {}
for dataset_id in dataset_ids['datasetId']:
    current_dataset_idx[dataset_id] = 0

#  Find all points that are within range of the starting point
for dataset_id in dataset_ids['datasetId']:
    point_dd  = [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                  points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ]
    point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                            points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
    
    #  Check distance from starting point
    geod = Geodesic.WGS84.Inverse( start_coord[0], start_coord[1], point_dd[0], point_dd[1] )
    dist = np.linalg.norm( point_utm - start_coord_utm )
    
    if dist < dist_thresh_m:
        
            #  Compute angle to new point
            temp_angles.append( geod['azi1'])
            avg_pt_utm += point_utm
            pt_count += 1

#  Compute seed point
avg_angle = mean_angle( temp_angles )
start_pt = Geodesic.WGS84.Direct( start_coord[0], start_coord[1], avg_angle, step_dist_m )
start_pt_utm = proj_dd2utm.transform( start_pt['lat2'], start_pt['lon2'] )

waypoint_list = [ [ np.array( [ start_pt['lat2'], start_pt['lon2'] ] ), 
                    np.array( [ start_pt_utm[0], start_pt_utm[1] ], np.float64 ),
                    avg_angle ] ]
display( waypoint_list )

#  Update our iterators past all points "behind" the coordinate
for dataset_id in dataset_ids['datasetId']:

    while True:
        point_dd  = np.array( [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                                points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ], np.float64 )
        point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                                points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
        
        # Update the indeces until the next coordinate is no longer in "front"
        if ( not Is_In_Front( waypoint_list[-1][0], point_dd, waypoint_list[-1][2] ) ) and ( np.linalg.norm( point_utm - waypoint_list[-1][1] ) < step_dist_m ):
            print('Current Dataset: {}, IDX: {}, Rejecting: {}'.format( dataset_id, current_dataset_idx[dataset_id], point_dd ))
            current_dataset_idx[dataset_id] += 1
        else:
            break
print(current_dataset_idx)    

NameError: name 'Geodesic' is not defined

## Step 5:  Compute the remaining route

In [ ]:
for i in range(0,600):
    temp_angles = []
    
    #  Update Current Indices
    for dataset_id in dataset_ids['datasetId']:
        point_dd  = [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                      points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ]
        point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                                points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
        
        #  Check distance from starting point
        geod = Geodesic.WGS84.Inverse( waypoint_list[-1][0][0], 
                                       waypoint_list[-1][0][1],
                                       point_dd[0],
                                       point_dd[1] )
        dist = np.linalg.norm( point_utm - waypoint_list[-1][1] )
    
        if dist < dist_thresh_m:
        
                #  Compute angle to new point
                temp_angles.append( geod['azi1'])

    #  Compute seed point
    avg_angle = mean_angle( temp_angles )
    next_point_dd = Geodesic.WGS84.Direct( waypoint_list[-1][0][0], 
                                           waypoint_list[-1][0][1],
                                           avg_angle, 
                                           step_dist_m )
    next_point_utm = proj_dd2utm.transform( next_point_dd['lat2'], next_point_dd['lon2'] )

    waypoint_list.append( [ np.array( [ next_point_dd['lat2'], next_point_dd['lon2'] ] ), 
                            np.array( [ next_point_utm[0], next_point_utm[1] ], np.float64 ),
                            avg_angle ] )
    
    #  Update our iterators past all points "behind" the coordinate
    for dataset_id in dataset_ids['datasetId']:

        while True:
            point_dd  = np.array( [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                                    points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ], np.float64 )
            point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                                    points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
        
            # Update the indeces until the next coordinate is no longer in "front"
            if ( not Is_In_Front( waypoint_list[-1][0], point_dd, waypoint_list[-1][2] ) ) and ( np.linalg.norm( point_utm - waypoint_list[-1][1] ) < step_dist_m ):
                current_dataset_idx[dataset_id] += 1
            else:
                break  


## Visualize Results

In [ ]:
centroid_pt = [0, 0]

#  Create the polyline and list of points
marker_list = [[start_coord[0],start_coord[1]]]
polyline   = []

for point in waypoint_list:
    marker_list.append( [point[0][0],point[0][1]] )
    centroid_pt[0] += point[0][0]
    centroid_pt[1] += point[0][1]
centroid_pt[0] /= len( waypoint_list )
centroid_pt[1] /= len( waypoint_list )
print( centroid_pt )

#  Build Map Visualization    
sector_map = Map( center=centroid_pt, zoom=14 )

#---------------------------------#
#-      Print the datasets       -#
#---------------------------------#
dataset_points = []
dataset_counter = 0.0
for dataset_id in dataset_ids['datasetId']:
    dataset_route = []
    for pidx in range( 0, len( points_by_dataset[dataset_id]['points']['latitude'] ) ):
        test_point = [ points_by_dataset[dataset_id]['points']['latitude'][pidx],
                       points_by_dataset[dataset_id]['points']['longitude'][pidx] ]
        #dataset_points.append( Marker( location=test_point, 
        #                               draggable=False,
        #                               color=matplotlib.colors.rgb2hex( plt.get_cmap('hsv')( dataset_counter / len(dataset_ids['datasetId']) ) ) ) )
        dataset_route.append( test_point )
    sector_map.add_layer( Polyline( locations= dataset_route,
                                    color=matplotlib.colors.rgb2hex( plt.get_cmap('hsv')( dataset_counter / len(dataset_ids['datasetId']) ) ),
                                    fill=False ) )
    
for p in dataset_points:
    sector_map.add_layer( p )
    
#-------------------------------------#
#-      Print the average route      -#
#-------------------------------------#
sector_map.add_layer( Marker( location=[start_coord[0],start_coord[1]] ) )
#for marker in marker_list:
#    marker = Marker( location=marker, draggable=False )
#    sector_map.add_layer( marker )
   
route_poly = Polyline( locations=marker_list,
                       color='blue',
                       fill=False )
sector_map.add_layer( route_poly )
    
#sector_map.layout.height="400px"
sector_map